In [1]:
import os
import wave
import numpy as np
import librosa
from utils import endpoint_detection
from tqdm import tqdm
import random

def extract_mfcc(file_path):
    """
    提取MFCC特征
    """
    with wave.open(file_path, 'rb') as wf:
        nframes = wf.getnframes()
        rate = wf.getframerate()
        strData = wf.readframes(nframes)
        waveData = np.frombuffer(strData, dtype=np.int16)

    # 端点检测
    s, e = endpoint_detection(waveData)
    signal = waveData[s: e]*1.0
    mfccs = librosa.feature.mfcc(y=signal, sr=rate, n_mfcc=40, n_mels=128, n_fft=256, hop_length=80, win_length=256, lifter=12)
    mean = np.mean(mfccs, axis=1, keepdims=True)
    std = np.std(mfccs, axis=1, keepdims=True)
    fea = (mfccs-mean)/std
    # 添加一阶差分和二阶差分
    fea_d = librosa.feature.delta(fea, order=1)
    fea_dd = librosa.feature.delta(fea, order=2)
    fea = np.concatenate((fea.T, fea_d.T, fea_dd.T), axis=1)
    return fea


def extract_mfcc2(file_path):
    """
    提取MFCC特征
    """
    with wave.open(file_path, 'rb') as wf:
        nframes = wf.getnframes()
        rate = wf.getframerate()
        strData = wf.readframes(nframes)
        waveData = np.frombuffer(strData, dtype=np.int16)

    # 端点检测
    s, e = endpoint_detection(waveData)
    signal = waveData[s: e]*1.0

    mel_spec = librosa.feature.melspectrogram(y=signal, sr=rate, n_mels=40, n_fft=256, hop_length=128, win_length=256, power=2.0)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

    # 标准化
    mean = np.mean(log_mel_spec, axis=1, keepdims=True)
    std = np.std(log_mel_spec, axis=1, keepdims=True)
    fea = (log_mel_spec - mean) / (std + 1e-6)

    # 添加一阶差分、二阶差分
    fea_d = librosa.feature.delta(fea, order=1)
    fea_dd = librosa.feature.delta(fea, order=2)
    fea = np.concatenate((fea.T, fea_d.T, fea_dd.T), axis=1)
    # print(fea.shape)
    # print(fea[0])
    return fea

In [6]:
# 测试部分
models = np.load("models_hmmlearn_0603.npy", allow_pickle=True)

# val_sids = []
sids = [sid for sid in os.listdir("dataset")]

def calculate_accuracy(sid):
    count = 0
    base_path = os.path.join("dataset", sid)
    files = os.listdir(base_path)
    data_paths = [os.path.join(base_path, file) for file in files]
    wrong_cnt = {"01": 0, "02": 0, "03": 0, "04": 0, "05": 0, "06": 0, "07": 0, "08": 0, "09": 0, "10": 0, 
                  "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "18": 0, "19": 0, "20": 0}

    for file_path in tqdm(data_paths):
        id, word, cnt = os.path.basename(file_path).split("_")
        fea = extract_mfcc(file_path)
        label_true = int(word)
        scores = []
        for m in range(20):
            model = models[m]
            score, _ = model.decode(fea)
            scores.append(score)
        # print(scores)

        # 根据分值进行识别
        det_lab = np.argmax(scores)+1
        if det_lab == label_true:
            count = count+1
        else:
            wrong_cnt[word] += 1
            # print("true lab %d det lab %d" % (label_true, det_lab))
    print("%s : accuracy %.2f %%" % (sid, count*100/len(data_paths)))
    print("wrong cnt: ", wrong_cnt)

for sid in sids:
    calculate_accuracy(sid)
    # break

  0%|          | 0/400 [00:00<?, ?it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  1%|          | 4/400 [00:00<00:22, 17.32it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  2%|▏         | 7/400 [00:00<00:18, 20.90it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  2%|▎         | 10/400 [00:00<00:20, 18.70it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  3%|▎         | 13/400 [00:00<00:24, 16.00it/s]Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 15/400 [00:00<00:28, 13.71it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 18/400 [00:01<00:22, 16.69it/s]Degenerate mixture covariance
Degenerate mixture covariance
  5%|▌         | 20/400 [00:01<00:26, 14.40it/s]Degenerate mixture covarian

21307110148 : accuracy 70.00 %
wrong cnt:  {'01': 1, '02': 2, '03': 1, '04': 2, '05': 4, '06': 0, '07': 16, '08': 1, '09': 0, '10': 16, '11': 7, '12': 8, '13': 8, '14': 3, '15': 7, '16': 0, '17': 17, '18': 6, '19': 18, '20': 3}


  0%|          | 0/400 [00:00<?, ?it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  1%|          | 4/400 [00:00<00:13, 28.93it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  2%|▏         | 7/400 [00:00<00:13, 28.93it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  3%|▎         | 11/400 [00:00<00:13, 29.92it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 15/400 [00:00<00:12, 31.75it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  5%|▍         | 19/400 [00:00<00:12, 31.38it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  6%|▌         | 23/400 [00:00<00

21307110234 : accuracy 57.50 %
wrong cnt:  {'01': 6, '02': 0, '03': 4, '04': 11, '05': 11, '06': 4, '07': 14, '08': 7, '09': 0, '10': 10, '11': 20, '12': 18, '13': 20, '14': 4, '15': 7, '16': 12, '17': 2, '18': 5, '19': 4, '20': 11}


  0%|          | 0/400 [00:00<?, ?it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  1%|          | 4/400 [00:00<00:11, 33.52it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  2%|▏         | 8/400 [00:00<00:11, 33.89it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  3%|▎         | 12/400 [00:00<00:11, 34.09it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 16/400 [00:00<00:11, 34.37it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  5%|▌         | 20/400 [00:00<00:11, 34.19it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  6

...

22307130313 : accuracy 95.75 %
wrong cnt:  {'01': 0, '02': 4, '03': 0, '04': 0, '05': 0, '06': 0, '07': 0, '08': 6, '09': 0, '10': 0, '11': 3, '12': 2, '13': 0, '14': 0, '15': 0, '16': 2, '17': 0, '18': 0, '19': 0, '20': 0}


  0%|          | 0/400 [00:00<?, ?it/s]Degenerate mixture covariance
Degenerate mixture covariance
  0%|          | 2/400 [00:00<00:37, 10.70it/s]Degenerate mixture covariance
Degenerate mixture covariance
  1%|▏         | 5/400 [00:00<00:51,  7.69it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  2%|▏         | 8/400 [00:00<00:30, 12.67it/s]Degenerate mixture covariance
Degenerate mixture covariance
  2%|▎         | 10/400 [00:00<00:31, 12.40it/s]Degenerate mixture covariance
Degenerate mixture covariance
  3%|▎         | 12/400 [00:01<00:40,  9.48it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 15/400 [00:01<00:29, 13.13it/s]Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 17/400 [00:01<00:30, 12.42it/s]Degenerate mixture covariance
Degenerate mixture covariance
  5%|▍         | 19/400 [00:01<00:32, 11.90it/s]Degenerate mixture covariance
Degenerate

22307130505 : accuracy 76.00 %
wrong cnt:  {'01': 8, '02': 1, '03': 1, '04': 6, '05': 7, '06': 6, '07': 9, '08': 10, '09': 5, '10': 1, '11': 5, '12': 19, '13': 0, '14': 0, '15': 0, '16': 1, '17': 11, '18': 0, '19': 4, '20': 2}


  0%|          | 0/400 [00:00<?, ?it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  1%|          | 4/400 [00:00<00:12, 31.52it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  2%|▏         | 8/400 [00:00<00:14, 27.49it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  3%|▎         | 13/400 [00:00<00:11, 32.28it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  4%|▍         | 17/400 [00:00<00:11, 33.43it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
  5%|▌         | 21/400 [00:00<00:11, 33.64it/s]Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Deg

23300240026 : accuracy 43.25 %
wrong cnt:  {'01': 5, '02': 10, '03': 20, '04': 0, '05': 19, '06': 0, '07': 0, '08': 4, '09': 0, '10': 20, '11': 18, '12': 20, '13': 19, '14': 0, '15': 15, '16': 3, '17': 20, '18': 19, '19': 20, '20': 15}
